In [6]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd

In [18]:
load_dotenv()

True

In [19]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [20]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
embedding = OpenAIEmbeddings()

In [22]:
ASTRA_DB_API_ENDPOINT='https://8a9d3acd-528e-4be1-bb1e-1b17b4cf9399-us-east-2.apps.astra.datastax.com'

In [23]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:ZtIaEuyRFKTfHvQSNOqkYhag:05784d026f2f4f3933cf881a7dcfa13b54354b712f38280921fbbd37bdf51a6d"

In [24]:
ASTRA_DB_KEYSPACE= "default_keyspace"

In [25]:
collection_name= "financebot"

In [26]:
vstore = AstraDBVectorStore(
    embedding= embedding,
    collection_name= collection_name,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE
    )